In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:77% !important; }</style>"))
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [45]:
import os
import re
import pickle
import torch
import torchvision
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd
from collections import OrderedDict
from net.st_gcn import Model
from feeder.feeder import Feeder
from ad_utils import map2ind, loader_initializer
from torchlight.io import IO
np.set_printoptions(linewidth=100, precision=5, suppress=True)

In [4]:
graph_args = {'layout':'openpose', 'strategy':'spatial'}

In [5]:
# Loading of original weights
root_path = '/root/sharedfolder/Research/pose_ad/st-gcn/'
io = IO(root_path)
model_name = 'net.st_gcn.Model'
model_args = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':400, }
model_args_ft = {
    'edge_importance_weighting':True,
    'graph_args':graph_args,
    'in_channels':3,
    'num_class':4, }

In [6]:
model_fio = io.load_model(model_name, **model_args)
model_2ft = io.load_model(model_name, **model_args_ft)

In [7]:
weights_path =  os.path.join(root_path, 'models/kinetics-st_gcn.pt')

In [28]:
weights = torch.load(weights_path)
weights_list = [[k.split('module.')[-1], v.cpu()] for k, v in weights.items()]
weights = OrderedDict(weights_list)
model_state_dict = model_2ft.state_dict()
weights.pop('fcn.bias') # loading all but the Final FC layer's weight and bias
weights.pop('fcn.weight')
model_state_dict.update(weights)
model_2ft.load_state_dict(model_state_dict)

In [10]:
data_dir_path = '/root/sharedfolder/datasets/data_ssd/kinetics-skeleton/st-gcn_kinetics/Kinetics/kinetics-skeleton/'
train_data_path = os.path.join(data_dir_path, 'train_data.npy')
train_label_path = os.path.join(data_dir_path, 'train_label.pkl')
test_data_path = os.path.join(data_dir_path, 'val_data.npy')
test_label_path = os.path.join(data_dir_path, 'val_label.pkl')

test_feeder_args = {'data_path': test_data_path, 'label_path': test_label_path}
train_feeder_args = {'data_path': train_data_path, 'label_path': train_label_path}

batch_size=32
data_loader = dict()
data_loader['kinetics_train'] = loader_initializer(train_feeder_args, batch_size=batch_size)
data_loader['kinetics_test' ] = loader_initializer(test_feeder_args, batch_size=batch_size)

In [11]:
train_npy = np.load(train_data_path, mmap_mode='r')
test_npy = np.load(test_data_path, mmap_mode='r')

In [12]:
save_files = False
gen_train = False
if gen_train:
    split_npy = train_npy
    split_2gen = 'normal_train'
    f = train_label_path
else:
    split_npy = test_npy
    split_2gen = 'normal_test'
    f = test_label_path

In [13]:
sample_name, label = pickle.load(open(f, "rb"))

In [14]:
data_path = dict()
label_path = dict()
split_indices = dict()
data_npy = dict()
data_pkl = dict()  # An array of [sample_name, label]


In [15]:
# Train Split Generator
data_pkl['normal_train'] = [[],[]]  # Important to zero so not to accumulate older values
data_pkl['normal_test'] = [[],[]]
for split in ['front_raises134', 'deadlifting88', 'clean_jerk59', 'forcast254', 'normal_train', 'normal_test', 'abnormal_test', 'mixed_test']:
    data_path[split]  =  os.path.join(data_dir_path, 'ad_experiment', split+'_data.npy')
    label_path[split] =  os.path.join(data_dir_path, 'ad_experiment', split+'_label.pkl')
    split_num = re.findall('\d+', split)
    if split_num != []:
        split_indices[split] = [ind for ind, val in enumerate(label) if val == int(split_num[0])]
        data_npy[split] = split_npy[split_indices[split]]
        data_pkl[split] = [[sample_name[i] for i in split_indices[split]], [label[i] for i in split_indices[split]]]
        data_pkl[split_2gen][0] += data_pkl[split][0]
        data_pkl[split_2gen][1] += data_pkl[split][1]
        print("Split {}, data_pkl[split][0] len={}, data_pkl[split_2gen][0] len={}".format(split, len(data_pkl[split][0]), len(data_pkl[split_2gen][0])))

data_npy[split_2gen] = np.concatenate((data_npy['front_raises134'], data_npy['deadlifting88'], data_npy['clean_jerk59'], data_npy['forcast254']))

Split front_raises134, data_pkl[split][0] len=50, data_pkl[split_2gen][0] len=50
Split deadlifting88, data_pkl[split][0] len=49, data_pkl[split_2gen][0] len=99
Split clean_jerk59, data_pkl[split][0] len=49, data_pkl[split_2gen][0] len=148
Split forcast254, data_pkl[split][0] len=50, data_pkl[split_2gen][0] len=198


In [16]:
print(data_npy['normal_test'].shape, len(data_pkl['normal_test'][0]))

(198, 3, 300, 18, 2) 198


In [20]:
if save_files and gen_train:
    np.save(data_path['normal_train'], data_npy['normal_train'])
    pickle.dump(data_pkl['normal_train'], open(label_path['normal_train'] ,"wb"))
elif save_files:
    np.save(data_path['normal_test'], data_npy['normal_test'])
    pickle.dump(data_pkl['normal_test'], open(label_path['normal_test'] ,"wb"))

In [17]:
# test_feeder_args = {'data_path': test_data_path, 'label_path': test_label_path}
normal_train_feeder_args = {'data_path': data_path['normal_train'], 'label_path': label_path['normal_train']}
normal_test_feeder_args = {'data_path': data_path['normal_test'], 'label_path': label_path['normal_test']}
mixed_test_feeder_args = {'data_path': data_path['mixed_test'], 'label_path': label_path['mixed_test']}

batch_size=32
data_loader['normal_train'] = loader_initializer(normal_train_feeder_args, batch_size=batch_size, num_workers=0)
data_loader['normal_test' ] = loader_initializer(normal_test_feeder_args, batch_size=batch_size, num_workers=0)
data_loader['mixed_test' ] = loader_initializer(mixed_test_feeder_args, batch_size=batch_size, suffle=False, drop_last=False)

In [18]:
dev = 'cuda:0'
model = model_2ft.cuda()
loss_fn = nn.CrossEntropyLoss()
loader = data_loader['normal_train']

In [25]:
model.st_gcn_networks[9]

st_gcn(
  (gcn): ConvTemporalGraphical(
    (conv): Conv2d(256, 768, kernel_size=(1, 1), stride=(1, 1))
  )
  (tcn): Sequential(
    (0): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (1): ReLU(inplace)
    (2): Conv2d(256, 256, kernel_size=(9, 1), stride=(1, 1), padding=(4, 0))
    (3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (4): Dropout(p=0, inplace)
  )
  (relu): ReLU(inplace)
)

In [27]:
# Freezing all but the new layer
for name, param in model.named_parameters():
    param.requires_grad = False
    if "fcn" in name:
        param.requires_grad = True

# Opens st-gcn layer 9 for learning
# model.st_gcn_networks[9].requires_grad = True
for name, param in model.named_parameters():
#     if name.find('9') != -1:
#         param.requires_grad = True
    print (name, param.requires_grad) 

data_bn.weight False
data_bn.bias False
st_gcn_networks.0.gcn.conv.weight False
st_gcn_networks.0.gcn.conv.bias False
st_gcn_networks.0.tcn.0.weight False
st_gcn_networks.0.tcn.0.bias False
st_gcn_networks.0.tcn.2.weight False
st_gcn_networks.0.tcn.2.bias False
st_gcn_networks.0.tcn.3.weight False
st_gcn_networks.0.tcn.3.bias False
st_gcn_networks.1.gcn.conv.weight False
st_gcn_networks.1.gcn.conv.bias False
st_gcn_networks.1.tcn.0.weight False
st_gcn_networks.1.tcn.0.bias False
st_gcn_networks.1.tcn.2.weight False
st_gcn_networks.1.tcn.2.bias False
st_gcn_networks.1.tcn.3.weight False
st_gcn_networks.1.tcn.3.bias False
st_gcn_networks.2.gcn.conv.weight False
st_gcn_networks.2.gcn.conv.bias False
st_gcn_networks.2.tcn.0.weight False
st_gcn_networks.2.tcn.0.bias False
st_gcn_networks.2.tcn.2.weight False
st_gcn_networks.2.tcn.2.bias False
st_gcn_networks.2.tcn.3.weight False
st_gcn_networks.2.tcn.3.bias False
st_gcn_networks.3.gcn.conv.weight False
st_gcn_networks.3.gcn.conv.bias False


In [21]:
# # loading files manually for verifying the loader
# fdata = np.load(normal_train_feeder_args['data_path'], mmap_mode='r')
# fsample_name, flabel = pickle.load(open(normal_train_feeder_args['label_path'], "rb"))

# fdata = np.load(normal_test_feeder_args['data_path'], mmap_mode='r')
fsample_name, flabel = pickle.load(open(normal_test_feeder_args['label_path'], "rb"))

print(len(fsample_name), len(flabel))

198 198


In [28]:
# Sanity Check - Using the original data 
# model = model_fio.cuda()
# loss_fn = nn.CrossEntropyLoss()
# loader = data_loader['kinetics_train']

In [31]:
optimizer = optim.Adam(model.parameters())
# optimizer = optim.SGD(model.parameters(), lr=1e-4, momentum=0.9)

In [39]:
# Training Loop
for epoch in range(2):
    for itern, [data, label] in enumerate(loader):
            # get data
            data = data.float().to(dev)
            label = torch.from_numpy(map2ind(label))
            label = label.long().to(dev)

            # forward
            output = model(data)
            loss = loss_fn(output, label)

            # backward
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            if itern%10 == 0: 
                print("Iteration {} (Epoch: {}), loss is {}".format(itern, epoch, loss))
            if loss < 0.15:
                print("Met loss {} in Epoch {}, iteration {}".format(loss, epoch, itern))
                break
    else:
        continue
    break
print("Done")

Iteration 0 (Epoch: 0), loss is 0.2892225980758667
Iteration 10 (Epoch: 0), loss is 0.24911516904830933
Met loss 0.12788908183574677 in iteration 13
Done


In [40]:
sfmax_vals, cls = torch.max(output, 1)
print(cls==label)
softmax = nn.Softmax(dim=1)
osoftmax_gpu = softmax(output)

sfmax_vals, cls = torch.max(osoftmax_gpu, 1)

osoftmax = 100 * osoftmax_gpu.cpu().detach().numpy()
print(osoftmax, '\n', sfmax_vals)
torch.mean(sfmax_vals)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0', dtype=torch.uint8)
[[99.28692  0.34906  0.16467  0.19935]
 [ 2.62818  1.21594  0.54397 95.61191]
 [98.52151  0.49589  0.71857  0.26402]
 [ 0.6491   0.09719 99.24773  0.00597]
 [ 7.56669  2.93043  4.22065 85.28223]
 [16.08512  9.81683  7.16591 66.93213]
 [ 3.24515  0.43408  0.33628 95.98448]
 [ 4.86326  3.26681  2.09784 89.77209]
 [ 0.99868 96.28539  2.68779  0.02814]
 [ 4.33689 78.13078 17.17232  0.36001]
 [ 8.10341 60.57731 31.21624  0.10304]
 [ 5.69339  5.16186  3.32176 85.82299]
 [ 0.06614  0.01862 99.89435  0.0209 ]
 [ 5.92181  6.13845  3.27562 84.66412]
 [97.62436  0.29696  0.12541  1.95328]
 [21.95343 64.76116 12.34718  0.93823]
 [ 4.93856  2.50877 92.4697   0.08298]
 [10.85381  0.87585  1.79512 86.47523]
 [13.34172 79.11286  7.29128  0.25415]
 [98.15436  0.48274  0.99597  0.36692]
 [ 4.62681  9.60966 83.60063  2.1629 ]
 [ 0.60818  0.16174  0.20115 99.

tensor(0.8866, device='cuda:0', grad_fn=<MeanBackward1>)

In [41]:
# Test loop
model.eval()
loader = data_loader['mixed_test']
loss_value = []
result_frag = []
label_frag = []
epoch_info = dict()
confusion_mat = np.zeros([5,5], dtype=np.int32)
evaluation = True

In [ ]:
# tt_data = np.load(data_path['mixed_test'])
# fpkl = '/root/sharedfolder/datasets/data_ssd/kinetics-skeleton/st-gcn_kinetics/Kinetics/kinetics-skeleton/ad_experiment/mixed_test_label.pkl'
# fpkl_names, fpkl_labels = pickle.load(open(fpkl ,"rb"))

In [42]:
for itern, [data, label] in enumerate(loader):
    print (data.shape, len(label))
    data = data.float().cuda()
    label = torch.from_numpy(map2ind(label, from_arr=[59, 88, 134, 254], to_arr=None, def_val=4))
    print(label)


torch.Size([32, 3, 300, 18, 2]) 32
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
        3, 3, 3, 3, 3, 3, 3, 3])
torch.Size([32, 3, 300, 18, 2]) 32
tensor([3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 4, 

In [44]:
for itern, [data, label] in enumerate(loader):
    # get data
    print(data.shape)
    data = data.float().cuda()
    label_mapped = map2ind(label, from_arr=[59, 88, 134, 254], to_arr=None, def_val=4)
    label = torch.from_numpy(label_mapped)
    label = label.long().cuda()

    # inference
    with torch.no_grad():
        output = model(data)
    result_frag.append(output.data.cpu().numpy())

    # get loss
    if evaluation:
        loss = loss_fn(output, label)
#         confusion_mat[label_mapped, np.argmax(output.cpu(), axis=1)] += 1
        loss_value.append(loss.item())
        label_frag.append(label.data.cpu().numpy())

result = np.concatenate(result_frag)
# np.savetxt("kinetics400_confusion.csv", confusion_mat)
if evaluation:
    label2 = np.concatenate(label_frag)
    epoch_info['mean_loss'] = np.mean(loss_value)
#     show_epoch_info()

    # show top-k accuracy
#     for k in self.arg.show_topk:
#         self.show_topk(k)

torch.Size([32, 3, 300, 18, 2])


RuntimeError: cuda runtime error (59) : device-side assert triggered at /pytorch/aten/src/THC/generic/THCTensorCopy.cpp:20